# Описание проекта

Добывающей компании нужно решить, где бурить новую скважину.

Для решения задачи предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Надо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой Bootstrap.

Шаги для выбора локации:

* В избранном регионе ищут месторождения, для каждого определяют значения признаков;
* Строят модель и оценивают объём запасов;
* Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
* Прибыль равна суммарной прибыли отобранных месторождений.

Условия задачи:

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

# 1. Загрузка и подготовка данных
Данные геологоразведки трёх регионов находятся в файлах:
* /datasets/geo_data_0.csv 
* /datasets/geo_data_1.csv 
* /datasets/geo_data_2.csv

* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей)

In [1]:
# Загрузка необходимых библиотек и таблиц с данными
import pandas as pd  
import numpy as np 
from scipy import stats as st

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error

df0 = pd.read_csv('geo_data_0.csv')
df1 = pd.read_csv('geo_data_1.csv')
df2 = pd.read_csv('geo_data_2.csv')

In [2]:
# Анализ исходных данных
df = [df0,df1,df2]
for i in df:
    print('Образец')
    display(i.head())
    print('Основные характеристики')
    print(i.describe())
    print('Пропуски в данных - ', df0.isnull().values.any())

Образец


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Основные характеристики
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
Пропуски в данных -  False
Образец


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


Основные характеристики
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%         1.153055      -4.813172       2.011479      57.085625
75%         8.621015      -1.332816       3.999904     107.813044
max        29.421755      18.734063       5.019721     137.945408
Пропуски в данных -  False
Образец


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Основные характеристики
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%         0.009424      -0.009482       2.484236      94.925613
75%         1.158535       1.163678       4.858794     130.595027
max         7.238262       7.844801      16.739402     190.029838
Пропуски в данных -  False


# Вывод
Все три таблицы исходных данных:
- имеют одинаковую структуру и размеры
- данные (features) не нуждаются в масштабировании
- целевой признак имеет распределение близкое к нормальному (среднее и медиана близки)
- столбец id не нужен для построения модели

# 2. Обучение и проверка модели

### 2.1. Разбиение данные на обучающую и валидационную выборки в соотношении 75:25.

In [2]:
# Удаляем не нужный столбец
df0 = df0.drop(['id'], axis=1).copy()
df1 = df1.drop(['id'], axis=1).copy()
df2 = df2.drop(['id'], axis=1).copy()
# Извлекаем признаки
features_df0 = df0.drop(['product'], axis=1)
features_df1 = df1.drop(['product'], axis=1)
features_df2 = df2.drop(['product'], axis=1)
# Извлекаем целевой признак
target_df0 = df0['product']
target_df1 = df1['product']
target_df2 = df2['product']

# Разбиваем выборку
features_train_df0, features_valid_df0, target_train_df0, target_valid_df0 = train_test_split(
    features_df0, target_df0, test_size=0.25, random_state=42) 
features_train_df1, features_valid_df1, target_train_df1, target_valid_df1 = train_test_split(
    features_df1, target_df1, test_size=0.25, random_state=42) 
features_train_df2, features_valid_df2, target_train_df2, target_valid_df2 = train_test_split(
    features_df2, target_df2, test_size=0.25, random_state=42) 

### 2.2. Обучение модели и предсказания на валидационной выборке.

In [3]:
model = LinearRegression()
model.fit(features_train_df0, target_train_df0) 
predict_valid_df0 = model.predict(features_valid_df0)
predict_valid_df0 = pd.Series(predict_valid_df0)
target_valid_df0 = target_valid_df0.reset_index(drop=True)
print('Регион 1')
print('Средний запас предсказанного сырья:', predict_valid_df0.mean())
print("RMSE модели линейной регрессии на валидационной выборке:",
      mean_squared_error(target_valid_df0, predict_valid_df0, squared=False))

Регион 1
Средний запас предсказанного сырья: 92.3987999065781
RMSE модели линейной регрессии на валидационной выборке: 37.75660035026169


In [4]:
model = LinearRegression()
model.fit(features_train_df1, target_train_df1) 
predict_valid_df1 = model.predict(features_valid_df1)
predict_valid_df1 = pd.Series(predict_valid_df1)
target_valid_df1 = target_valid_df1.reset_index(drop=True)
print('Регион 2')
print('Средний запас предсказанного сырья:', predict_valid_df1.mean())
print("RMSE модели линейной регрессии на валидационной выборке:",
      mean_squared_error(target_valid_df1, predict_valid_df1, squared=False))

Регион 2
Средний запас предсказанного сырья: 68.71287803913762
RMSE модели линейной регрессии на валидационной выборке: 0.8902801001028849


In [5]:
model = LinearRegression()
model.fit(features_train_df2, target_train_df2) 
predict_valid_df2 = model.predict(features_valid_df2)
predict_valid_df2 = pd.Series(predict_valid_df2)
target_valid_df2 = target_valid_df2.reset_index(drop=True)
print('Регион 3')
print('Средний запас предсказанного сырья:', predict_valid_df2.mean())
print("RMSE модели линейной регрессии на валидационной выборке:",
      mean_squared_error(target_valid_df2, predict_valid_df2, squared=False))

Регион 3
Средний запас предсказанного сырья: 94.77102387765996
RMSE модели линейной регрессии на валидационной выборке: 40.145872311342174


### Выводы
Наибольший средний запас предсказанного сырья в Регионе2.
Среднеквадратичная ошибка модели минимальна для модели по Региону 1

# 3. Подготовка к расчёту прибыли

In [7]:
# Данные для расчетов (суммы будут в млн. руб)
BUDGET = 10000 # Бюджет на разработку скважин в регионе
POINTS = 500 # Количество точек при разведке региона
BEST_POINTS = 200 # Количество лучших точек 
REVENUE_ONE = 0.45 # Доход с каждой единицы продукта
PROBABILITY_OF_LOSS = 0.025 # Граница вероятности убытков


In [8]:
print('Стоимость разработки одной скважины', BUDGET / BEST_POINTS, 'млн. руб')
print('Для этого необходимо добыть продукта', BUDGET / BEST_POINTS / REVENUE_ONE, 'единиц')

Стоимость разработки одной скважины 50.0 млн. руб
Для этого необходимо добыть продукта 111.11111111111111 единиц


Достаточный объём сырья для безубыточной разработки новой скважины - 111,11, при этом полученный объём сырья со средним запасом в регионах равен 92.40, 68.71, 94.77. Что меньше, чем безубыточный объем.

In [9]:
# Функция расчета прибыли
def revenue(target, probabilities, count):
    probabilities = pd.Series(probabilities)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return REVENUE_ONE * selected.sum() - BUDGET

### Вывод
* Достаточный объём сырья для безубыточной разработки новой скважины - 111,11 бареллей, что больше, чем объём сырья со средним запасом в регионах.
* Написана функция расчета прибыли, которая на вход принимает данные по скважинам, количество лучших  и возвращает суммарную прибыль по лучшим скважинам.

# 4. Расчёт прибыли и рисков

Напишем функцию для расчета основных параметров: средней выручки, доверительного интервала и вероятности получения убытка

In [12]:
# Функция бутстрапа 
def bootstrap(target, predict, points, best):
    state = np.random.RandomState(42) # случайный элемент
    values = [] # массив прибылей

    for i in range(1000): # Заполнение массива прибылей
        target_subsample = target.sample(n=points, replace=True, random_state=state)
        probs_subsample = pd.Series(predict[target_subsample.index])
        values.append(revenue(target_subsample, probs_subsample, best))
        
    values = pd.Series(values)
    print('Средняя выручка:', values.mean())
    print('95%-ый доверительный интервал:', pd.Series(values).quantile(0.025), pd.Series(values).quantile(0.975))
    print('Вероятность убытков:', values[values < 0].count() / len(values))

In [13]:
# Вывод основных параметров по регионам
print('Регион 1')
bootstrap(target_valid_df0, predict_valid_df0, POINTS, BEST_POINTS)
print('Регион 2')
bootstrap(target_valid_df1, predict_valid_df1, POINTS, BEST_POINTS)
print('Регион 3')
bootstrap(target_valid_df2, predict_valid_df2, POINTS, BEST_POINTS)

Регион 1
Средняя выручка: 427.847560462525
95%-ый доверительный интервал: -97.24982956859569 954.2151927088142
Вероятность убытков: 0.055
Регион 2
Средняя выручка: 511.3627761976858
95%-ый доверительный интервал: 98.87064990277922 940.7205116508032
Вероятность убытков: 0.009
Регион 3
Средняя выручка: 402.57560751419624
95%-ый доверительный интервал: -137.1622250719332 929.8875280253216
Вероятность убытков: 0.074


# Общий вывод

Для разработки скважин рекомендуется Регион 2, так как он имеет:
* наибольшую среднюю выручку
* минимальную вероятность убытков
* доверительный интервал в положительной зоне